In [1]:
import cv2

In [2]:
import cv2  # Import the OpenCV library for video processing

def extract_frames(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Initialize an empty list to store frames
    frames = []
    
    # Loop until the video file is open
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        
        # Check if the frame was read successfully
        if not ret:
            # Exit the loop if no more frames are available
            break
        
        # Append the frame to the list of frames
        frames.append(frame)
    
    # Release the video capture object
    cap.release()
    
    # Return the list of frames
    return frames

In [3]:
import cv2
import numpy as np

def run_yolo_detection(frame, net, output_layers):
    # Get the dimensions of the frame
    height, width = frame.shape[:2]
    
    # Prepare the frame for the YOLO model
    # Convert the image to a blob and normalize
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    
    # Set the input for the YOLO model
    net.setInput(blob)
    
    # Perform forward pass and get output from the output layers
    layer_outputs = net.forward(output_layers)
    
    # Initialize an empty list to store detected boxes
    boxes = []
    
    # Process each output layer
    for output in layer_outputs:
        for detection in output:
            # Extract class scores and class ID
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # Check if confidence is above the threshold
            if confidence > 0.5:
                # Calculate the bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Convert center coordinates and width/height to bounding box coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                # Append the bounding box coordinates to the list
                boxes.append([x, y, x + w, y + h])
                
    # Return the list of bounding boxes
    return boxes

In [4]:
def calculate_precision_over_frames(frames, ground_truths, net, output_layers, iou_threshold=0.5):
    # Initialize counters for true positives and false positives
    total_true_positives = 0
    total_false_positives = 0
    
    # Iterate over each frame and its corresponding ground truths
    for i, frame in enumerate(frames):
        # Run YOLO detection on the current frame
        detections = run_yolo_detection(frame, net, output_layers)
        
        # Retrieve the ground truths for the current frame
        frame_ground_truths = ground_truths[i]
        
        # Initialize counters for true positives and false positives for the current frame
        true_positives = 0
        false_positives = 0
        
        # Check each detected bounding box
        for detection in detections:
            matched = False
            # Compare detection with each ground truth
            for gt in frame_ground_truths:
                # Calculate Intersection over Union (IoU) between detection and ground truth
                iou = calculate_iou(detection, gt)
                if iou > iou_threshold:
                    # If IoU is above threshold, consider it a true positive
                    true_positives += 1
                    matched = True
                    break
            
            if not matched:
                # If no match was found, consider it a false positive
                false_positives += 1
        
        # Accumulate counts of true positives and false positives
        total_true_positives += true_positives
        total_false_positives += false_positives
    
    # Calculate overall precision
    overall_precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
    
    return overall_precision

In [5]:
def main(video_path, ground_truths):
    # Load YOLO model
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    # Get layer names
    layer_names = net.getLayerNames()

    # Handle both 1D and 2D outputs from getUnconnectedOutLayers
    unconnected_layers = net.getUnconnectedOutLayers()

    if isinstance(unconnected_layers[0], list):
        output_layers = [layer_names[i[0] - 1] for i in unconnected_layers]
    else:
        output_layers = [layer_names[i - 1] for i in unconnected_layers]
    
    frames = extract_frames(video_path)
    
    overall_precision = calculate_precision_over_frames(frames, ground_truths, net, output_layers)
    
    print(f"Overall Precision: {overall_precision:.2f}")

In [6]:
video_path = '4K Road traffic video for object detection and tracking - free download now!.mp4'